# Is Home Team gonna win?

In [304]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [305]:
df = pd.read_csv('data/dataset/2020-2024 Matches Liga 1 Teams.csv', parse_dates=['DateTime'])
df.drop(['Unnamed: 0'], axis=1, inplace=True)
# Se tomará solo la competición de la Liga 1
df = df.loc[df['Comp'] == 'Liga 1']
df.shape

(2211, 16)

In [306]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2211 entries, 0 to 2380
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           2211 non-null   object        
 1   Time           2211 non-null   object        
 2   Comp           2211 non-null   object        
 3   Round          2211 non-null   object        
 4   Day            2211 non-null   int64         
 5   Venue          2211 non-null   object        
 6   Result         2211 non-null   object        
 7   GF             2211 non-null   object        
 8   GA             2211 non-null   object        
 9   Opponent       2211 non-null   object        
 10  Captain        2151 non-null   object        
 11  Formation      2188 non-null   object        
 12  Opp Formation  2188 non-null   object        
 13  Referee        1800 non-null   object        
 14  Team           2211 non-null   object        
 15  DateTime       2211 non-nu

# Data Cleaning

In [307]:
# Eliminamos las columnas porque no ofrece información valiosa
df.drop(['Captain', 'Referee'], axis=1, inplace=True)

## Missing values

Imputaremos las formaciones más frequentes a cada equipo por año, para llenar los NaN

### Manejando NaN en 'Formation' 'Opp Formation'

In [308]:
# Encontramos que existen registros con una formación duplicada (4-4-2◆)
df['Formation'].unique()

array(['4-2-3-1', '4-4-2', '3-5-1-1', '3-5-2', '5-3-2', '3-4-3',
       '4-2-2-2', '4-3-3', '4-5-1', nan, '4-1-4-1', '5-4-1', '4-4-1-1',
       '3-4-1-2', '3-1-4-2', '4-1-3-2', '3-3-3-1', '4-4-2◆', '4-3-1-2',
       '4-3-2-1', '3-2-4-1', '4-2-4'], dtype=object)

In [309]:
df['Formation'] = df['Formation'].replace('4-4-2◆', value='4-4-2')
df['Formation'].unique()

array(['4-2-3-1', '4-4-2', '3-5-1-1', '3-5-2', '5-3-2', '3-4-3',
       '4-2-2-2', '4-3-3', '4-5-1', nan, '4-1-4-1', '5-4-1', '4-4-1-1',
       '3-4-1-2', '3-1-4-2', '4-1-3-2', '3-3-3-1', '4-3-1-2', '4-3-2-1',
       '3-2-4-1', '4-2-4'], dtype=object)

In [310]:
df['Opp Formation'].unique()

array(['4-2-3-1', '4-3-3', '4-4-2', '4-1-4-1', '3-5-2', '3-4-3',
       '3-4-1-2', '4-5-1', '4-4-1-1', '5-3-2', '5-4-1', nan, '3-1-4-2',
       '3-2-4-1', '4-3-1-2', '4-1-3-2', '4-2-4', '3-5-1-1', '4-2-2-2',
       '4-3-2-1', '3-3-3-1'], dtype=object)

In [311]:
# Por cada temporada de cada equipo, se le imputará la formación más frecuente
teams = df['Team'].unique()
for team in teams:
    years = df[df['Team'] == team]['DateTime'].dt.year.unique()
    for year in years:
        conditions = (df['DateTime'].dt.year == year) & (df['Team'] == team)
        # Si existe NaN de un equipo en un año
        # reemplazarlo por el valor que más se repite
        if df[conditions]['Formation'].isnull().any():
            mode = df[conditions]['Formation'].mode()
            df.loc[conditions, 'Formation'] = df[conditions]['Formation'].fillna(value=mode[0])
        # Hacer lo mismo con la formación del equipor rival
        if df[conditions]['Opp Formation'].isnull().any():
            mode = df[conditions]['Opp Formation'].mode()
            df.loc[conditions, 'Opp Formation'] = df[conditions]['Opp Formation'].fillna(value=mode[0])

In [312]:
print(f'Formation tiene NaN? {df['Formation'].isnull().any()}\nOpp Formation tiene NaN? {df['Opp Formation'].isnull().any()}')

Formation tiene NaN? False
Opp Formation tiene NaN? False


### Inconsistent Data Entry

In [313]:
df['GF'].unique()

array(['1', '4', '5', '0', '2', '3', '6', '7', '1 (3)', '1 (2)', '12'],
      dtype=object)

In [314]:
df.loc[df['GF'].str.len() > 2]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,Formation,Opp Formation,Team,DateTime
313,2020-12-05,14:00,Liga 1,Final,5,Away,D,1 (3),1 (2),Sporting Cristal,4-2-3-1,4-3-3,Ayacucho,2020-12-05 14:00:00
551,2020-12-05,14:00,Liga 1,Final,5,Home,D,1 (2),1 (3),Ayacucho,4-3-3,4-2-3-1,Sporting Cristal,2020-12-05 14:00:00


In [315]:
df.loc[df['GA'].str.len() > 2]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,Formation,Opp Formation,Team,DateTime
313,2020-12-05,14:00,Liga 1,Final,5,Away,D,1 (3),1 (2),Sporting Cristal,4-2-3-1,4-3-3,Ayacucho,2020-12-05 14:00:00
551,2020-12-05,14:00,Liga 1,Final,5,Home,D,1 (2),1 (3),Ayacucho,4-3-3,4-2-3-1,Sporting Cristal,2020-12-05 14:00:00


In [316]:
# Se tratará como empate y se ignorará los goles de penales
def get_goals(row):
    row['GF'] = row['GF'].split('(')[0].strip()
    row['GA'] = row['GA'].split('(')[0].strip()
    return row
df = df.apply(get_goals, axis=1)

In [317]:
df['GF'].unique()

array(['1', '4', '5', '0', '2', '3', '6', '7', '12'], dtype=object)

In [318]:
df['GA'].unique()

array(['2', '0', '1', '4', '3', '5', '6'], dtype=object)

In [319]:
df = df.astype({'GF': 'int16', 'GA': 'int16'})

In [320]:
df['GF'].unique()

array([ 1,  4,  5,  0,  2,  3,  6,  7, 12], dtype=int16)

# Baseline - Feature Engineering

Modificaremos el dataset para que los registros de partidos se reduzcan a un solo registro en vez de tener registro del mismo partido por equipo.

Por ejemplo: Partido Cantolao vs Cienciano
- Registro 1: Cantolao (Local)
- Registro 2: Cienciano (Visitante)

Ambos registros pertenecen al mismo partido

Objetivo:
- Registro: Cantolao (Local) vs Cienciano (Visitante)

In [321]:
df.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,Formation,Opp Formation,Team,DateTime
0,2020-02-02,13:00,Liga 1,Torneo Apertura,6,Away,L,1,2,Cantolao,4-2-3-1,4-2-3-1,Cienciano,2020-02-02 13:00:00
1,2020-02-09,15:00,Liga 1,Torneo Apertura,6,Home,W,4,0,USMP,4-2-3-1,4-3-3,Cienciano,2020-02-09 15:00:00
2,2020-02-14,15:00,Liga 1,Torneo Apertura,4,Home,W,5,2,Llacuabamba,4-4-2,4-4-2,Cienciano,2020-02-14 15:00:00
3,2020-02-24,20:00,Liga 1,Torneo Apertura,0,Away,L,0,1,Alianza Univ,4-4-2,4-3-3,Cienciano,2020-02-24 20:00:00
4,2020-02-29,15:30,Liga 1,Torneo Apertura,5,Home,L,1,4,Ayacucho,4-2-3-1,4-2-3-1,Cienciano,2020-02-29 15:30:00


In [322]:
df.shape

(2211, 14)

## Estandarizar nombre de equipos

In [323]:
all_teams = df.loc[:,['Opponent', 'Team']].values
all_teams

array([['Cantolao', 'Cienciano'],
       ['USMP', 'Cienciano'],
       ['Llacuabamba', 'Cienciano'],
       ...,
       ['Comerciantes Unidos', 'Sport Boys'],
       ['UTC', 'Sport Boys'],
       ['ADT Tarma', 'Sport Boys']], shape=(2211, 2), dtype=object)

In [324]:
teams = set([team for teams in all_teams for team in teams])
teams = sorted(teams)
print(teams)

['ADT Tarma', 'Alianza Atlético', 'Alianza Lima', 'Alianza Univ', 'Alianza Universidad', 'Atlético Grau', 'Ayacucho', 'Binacional', 'CDC Santa Rosa', 'Cantolao', 'Carlos Stein', 'Cienciano', 'Comerciantes Unidos', 'Cusco', 'Dep Municipal', 'Deportivo Garcilaso', 'Llacuabamba', 'Mannucci', 'Melgar', 'Sport Boys', 'Sport Huancayo', 'Sporting Cristal', 'U César Vallejo', 'USMP', 'UTC', 'Universidad Técnica de Cajamarca', 'Universitario', 'Unión Comercio']


In [325]:
from fuzzywuzzy import process
import fuzzywuzzy

for team in teams:
    matches = fuzzywuzzy.process.extract(team, teams, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    for t, score in matches:
        if score >= 75 and score != 100:
            print(team, t, score, sep=' - ')

Alianza Lima - Alianza Univ - 75
Alianza Univ - Alianza Universidad - 77
Alianza Univ - Alianza Lima - 75
Alianza Universidad - Alianza Univ - 77


Por lo expuesto, el único valor que debemos cambiar `Alianza Univ` por `Alianza Universidad` y `Universidad Técnica de Cajamarca` por `UTC`

In [326]:
df.loc[:, 'Opponent'] = df['Opponent'].replace({'Alianza Univ': 'Alianza Universidad', 'Universidad Técnica de Cajamarca': 'UTC'})
df.loc[:, 'Team'] = df['Team'].replace({'Alianza Univ': 'Alianza Universidad', 'Universidad Técnica de Cajamarca': 'UTC'})

## Ajustando features por partido

In [327]:
def get_result(row):
    if row['Venue'] == 'Home':
        if row['Result'] == 'W':
            row['Result'] = 0 # Win
        elif row['Result'] == 'D':
            row['Result'] = 1 # Draw
        else:
            row['Result'] = 2 # Lose
    else:
        if row['Result'] == 'W':
            row['Result'] = 2 # Lose
        elif row['Result'] == 'D':
            row['Result'] = 1 # Draw
        else:
            row['Result'] = 0 # Win

In [328]:
def get_match(row):
    if row['Venue'] == 'Home': # Team es el local
        row['Home Team'] = row['Team']
        row['Away Team'] = row['Opponent']
        get_result(row)
    elif row['Venue'] == 'Away': # Opponent es el local
        row['Home Team'] = row['Opponent']
        row['Away Team'] = row['Team']
        get_result(row)
        row['GF'], row['GA'] = row['GA'], row['GF']
        row['Formation'], row['Opp Formation'] = row['Opp Formation'], row['Formation']
    return row

In [329]:
df = (df.apply(get_match, axis=1)
      # Borramos los duplicados de partidos que comparten fecha, hora, equipo local y equipo visitante
      .drop_duplicates(subset=['DateTime', 'Home Team', 'Away Team'], ignore_index=True)
      # Borramos columnas que ya no aportan valor
      .drop(['Venue', 'Opponent', 'Team'], axis=1)
      # Ordenar las filas por fecha
      .sort_values(['DateTime'], axis=0, ignore_index=True)
     )

In [330]:
df.head()

,Date,Time,Comp,Round,Day,Result,GF,GA,Formation,Opp Formation,DateTime,Home Team,Away Team
0,2020-01-31,18:00,Liga 1,Torneo Apertura,4,2,1,2,4-3-3,4-2-3-1,2020-01-31 18:00:00,Melgar,Universitario
1,2020-01-31,20:30,Liga 1,Torneo Apertura,4,2,2,3,3-4-3,4-4-2,2020-01-31 20:30:00,Alianza Lima,Alianza Universidad
2,2020-02-01,15:00,Liga 1,Torneo Apertura,5,0,2,1,4-2-3-1,4-3-3,2020-02-01 15:00:00,UTC,Sporting Cristal
3,2020-02-01,17:30,Liga 1,Torneo Apertura,5,0,1,0,4-2-3-1,4-2-3-1,2020-02-01 17:30:00,Sport Huancayo,Atlético Grau
4,2020-02-02,11:00,Liga 1,Torneo Apertura,6,0,3,1,4-3-3,4-4-2,2020-02-02 11:00:00,USMP,Ayacucho


In [331]:
df.shape

(1416, 13)

## Analizar columna 'Round'
Verificamos que existe `Final` y `Finals` en la columna 'Round'

In [332]:
df['Round'].unique()

array(['Torneo Apertura', 'Group Stage', 'Final', 'Semi-finals', 'Finals',
       'Torneo Clausura'], dtype=object)

In [333]:
# Reemplazamos las filas con valor Finals con Final
df.loc[df['Round'] == 'Finals', 'Round'] = 'Final'

In [334]:
# Columnas del nuevo dataframe
columns = ['DateTime', 'Day', 'Time', 'Round', 'Home Team', 'Away Team', 'GF', 'GA', 'Formation', 'Opp Formation', 'Result']

In [335]:
df = df[columns]
df.head()

,DateTime,Day,Time,Round,Home Team,Away Team,GF,GA,Formation,Opp Formation,Result
0,2020-01-31 18:00:00,4,18:00,Torneo Apertura,Melgar,Universitario,1,2,4-3-3,4-2-3-1,2
1,2020-01-31 20:30:00,4,20:30,Torneo Apertura,Alianza Lima,Alianza Universidad,2,3,3-4-3,4-4-2,2
2,2020-02-01 15:00:00,5,15:00,Torneo Apertura,UTC,Sporting Cristal,2,1,4-2-3-1,4-3-3,0
3,2020-02-01 17:30:00,5,17:30,Torneo Apertura,Sport Huancayo,Atlético Grau,1,0,4-2-3-1,4-2-3-1,0
4,2020-02-02 11:00:00,6,11:00,Torneo Apertura,USMP,Ayacucho,3,1,4-3-3,4-4-2,0


## Goals Scored and Goals Conceded

In [336]:
seasons = df['DateTime'].dt.year.unique()
seasons

array([2020, 2021, 2022, 2023, 2024], dtype=int32)

In [337]:
"""
Por cada temporada, obtenemos los goles que los equipos de local y visitante 
han marcado a FAVOR. Estos goles serán guardados en las columnas 
'AHGS' (Accumulated Home Goals Scored) y 'AAGS' (Accumulated Away Goals Scored)
"""
goals_scored = dict()
for i in range(len(df)):
    hteam = df.loc[i, 'Home Team']
    ateam = df.loc[i, 'Away Team']
    season = df.loc[i, 'DateTime'].year

    # Si en el diccionario existe el equipo local en la temporada indicada
    # Se asigna el valor de goles acumulados del equipo.
    # Sino, se inicializa con 0
    if (season, hteam) in goals_scored.keys():
        df.loc[i, 'HGS'] = goals_scored[(season, hteam)]
    else:
        df.loc[i, 'HGS'] = 0
        goals_scored[(season, hteam)] = 0

    # Si en el diccionario existe el equipo visitante en la temporada indicada
    # Se asigna el valor de goles a FAVOR acumulados del equipo.
    # Sino, se inicializa con 0
    if (season, ateam) in goals_scored.keys():
        df.loc[i, 'AGS'] = goals_scored[(season, ateam)]
    else:
        df.loc[i, 'AGS'] = 0
        goals_scored[(season, ateam)] = 0

    # Se suma los goles a FAVOR del equipo local y visitante al diccionario
    # del partido en el índice i para asignarlo en la siguiente
    # iteración donde los equipos jueguen.
    goals_scored[(season, hteam)] += df.loc[i, 'GF']
    goals_scored[(season, ateam)] += df.loc[i, 'GA']

In [338]:
"""
Por cada temporada, obtenemos los goles que los equipos de local y visitante 
han marcado en CONTRA. Estos goles serán guardados en las columnas 
'HGC' (Home Goals Conceded) y 'AGC' (Away Goals Conceded)
"""
goals_conceded = dict()
for i in range(len(df)):
    hteam = df.loc[i, 'Home Team']
    ateam = df.loc[i, 'Away Team']
    season = df.loc[i, 'DateTime'].year

    # Si en el diccionario existe el equipo local en la temporada indicada
    # Se asigna el valor de goles en CONTRA acumulados del equipo.
    # Sino, se inicializa con 0
    if (season, hteam) in goals_conceded.keys():
        df.loc[i, 'HGC'] = goals_conceded[(season, hteam)]
    else:
        df.loc[i, 'HGC'] = 0
        goals_conceded[(season, hteam)] = 0

    if (season, ateam) in goals_conceded.keys():
        df.loc[i, 'AGC'] = goals_conceded[(season, ateam)]
    else:
        df.loc[i, 'AGC'] = 0
        goals_conceded[(season, ateam)] = 0
        
    goals_conceded[(season, hteam)] += df.loc[i, 'GA']
    goals_conceded[(season, ateam)] += df.loc[i, 'GF']

## Puntos acumulados por temporada y Fecha de partido

In [340]:
"""
Por cada temporada, obtenemos los puntos que los equipos de local y visitante 
acumulados. Estos puntos serán guardados en las columnas 
'AHP' (Home Teams Points) y 'AAP' (Away Teams Points)
"""
points_accumulated = dict()
match_week = dict()
for i in range(len(df)):
    season = df.loc[i, 'DateTime'].year
    hteam = df.loc[i, 'Home Team']
    ateam = df.loc[i, 'Away Team']

    if (season, hteam) in match_week.keys():
        match_week[(season, hteam)] += 1
        df.loc[i, 'MW'] = match_week[(season, hteam)]
    else:
        match_week[(season, hteam)] = 1
        df.loc[i, 'MW'] = 1

    if (season, ateam) in match_week.keys():
        match_week[(season, ateam)] += 1
        df.loc[i, 'MW'] = match_week[(season, ateam)]
    else:
        match_week[(season, ateam)] = 1
        df.loc[i, 'MW'] = 1

    if (season, hteam) in points_accumulated.keys():
        df.loc[i, 'HTP'] = points_accumulated[(season, hteam)]
    else:
        df.loc[i, 'HTP'] = 0
        points_accumulated[(season, hteam)] = 0

    if (season, ateam) in points_accumulated.keys():
        df.loc[i, 'ATP'] = points_accumulated[(season, ateam)]
    else:
        df.loc[i, 'ATP'] = 0
        points_accumulated[(season, ateam)] = 0

    if (df.loc[i, 'Result'] == 0): # Home Win
        points_accumulated[(season, hteam)] += 3
    elif (df.loc[i, 'Result'] == 1): # Draw
        points_accumulated[(season, hteam)] += 1
        points_accumulated[(season, ateam)] += 1
    else: # Home Lose
        points_accumulated[(season, ateam)] += 3

## Racha de partidos sin perder

In [342]:
undefeated_streak = dict()
for i in range(len(df)):
    hteam = df.loc[i, 'Home Team']
    ateam = df.loc[i, 'Away Team']

    if hteam in undefeated_streak.keys():
        df.loc[i, 'HUS'] = undefeated_streak[hteam]
    else:
        undefeated_streak[hteam] = 0
        df.loc[i, 'HUS'] = 0
    
    if ateam in undefeated_streak.keys():
        df.loc[i, 'AUS'] = undefeated_streak[ateam]
    else:
        undefeated_streak[ateam] = 0
        df.loc[i, 'AUS'] = 0

    if df.loc[i, 'Result'] == 0:
        undefeated_streak[hteam] += 1
        undefeated_streak[ateam] = 0
    elif df.loc[i, 'Result'] == 1:
        undefeated_streak[hteam] += 1
        undefeated_streak[ateam] += 1
    elif df.loc[i, 'Result'] == 2:
        undefeated_streak[hteam] = 0
        undefeated_streak[ateam] += 1

## Racha de derrotas

In [344]:
losing_streak = dict()
for i in range(len(df)):
    hteam = df.loc[i, 'Home Team']
    ateam = df.loc[i, 'Away Team']

    if hteam in losing_streak.keys():
        df.loc[i, 'HLS'] = losing_streak[hteam]
    else:
        losing_streak[hteam] = 0
        df.loc[i, 'HLS'] = 0

    if ateam in losing_streak.keys():
        df.loc[i, 'ALS'] = losing_streak[ateam]
    else:
        losing_streak[ateam] = 0
        df.loc[i, 'ALS'] = 0

    result = df.loc[i, 'Result']
    if result == 0: # Local win
        losing_streak[hteam] = 0
        losing_streak[ateam] += 1
    elif result == 1: # Draw
        losing_streak[hteam] = 0
        losing_streak[ateam] = 0
    elif result == 2: # Local Lose
        losing_streak[hteam] += 1
        losing_streak[ateam] = 0

## Diferencia de Goles

In [346]:
df['HGD'] = df['HGS'] - df['HGC']
df['AGD'] = df['AGS'] - df['AGC']

## Diferencia de puntos

In [348]:
df['PD'] = df['HTP'] - df['ATP']

In [350]:
df.shape

(1416, 25)

In [353]:
pd.set_option("display.max_columns", None)
df.head(20)

,DateTime,Day,Time,Round,Home Team,Away Team,GF,GA,Formation,Opp Formation,Result,HGS,AGS,HGC,AGC,MW,HTP,ATP,HUS,AUS,HLS,ALS,HGD,AGD,PD
0,2020-01-31 18:00:00,4,18:00,Torneo Apertura,Melgar,Universitario,1,2,4-3-3,4-2-3-1,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-31 20:30:00,4,20:30,Torneo Apertura,Alianza Lima,Alianza Universidad,2,3,3-4-3,4-4-2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-02-01 15:00:00,5,15:00,Torneo Apertura,UTC,Sporting Cristal,2,1,4-2-3-1,4-3-3,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-02-01 17:30:00,5,17:30,Torneo Apertura,Sport Huancayo,Atlético Grau,1,0,4-2-3-1,4-2-3-1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-02 11:00:00,6,11:00,Torneo Apertura,USMP,Ayacucho,3,1,4-3-3,4-4-2,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2020-02-02 13:00:00,6,13:00,Torneo Apertura,Cantolao,Cienciano,2,1,4-2-3-1,4-2-3-1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2020-02-02 15:30:00,6,15:30,Torneo Apertura,Sport Boys,Llacuabamba,3,2,4-3-3,4-2-3-1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2020-02-03 15:00:00,0,15:00,Torneo Apertura,Cusco,Binacional,0,2,4-4-2,4-2-3-1,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2020-02-07 15:00:00,4,15:00,Torneo Apertura,Ayacucho,Sport Boys,2,1,4-4-2,4-2-3-1,0,1.0,3.0,3.0,2.0,2.0,0.0,3.0,0.0,1.0,1.0,0.0,-2.0,1.0,-3.0
9,2020-02-08 13:15:00,5,13:15,Torneo Apertura,Atlético Grau,U César Vallejo,1,1,4-2-3-1,4-2-3-1,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0


# Visualizar datos